# Project: kedro_iris_ml_ops

### Author: Dale Jacques
### Contact: djacques@uwalumni.com
### Repository: https://github.com/AIgenVectorLabs/kedro_iris_ml_ops
#### Description: This notebook demonstrates concepts like the Kedro "Session", "Context", and "Catalog" that will be required when deploying our model into production. Specifically, it shows how a Kedro pipeline can be parameterized, run, and results can be extracted. 

In [5]:
# Load the context, then load and examine our training set from the catalog 
my_context = session.load_context()
my_context.io.load("example_iris_data")

2021-08-08 15:01:43,454 - kedro.io.data_catalog - INFO - Loading data from `example_iris_data` (CSVDataSet)...


/Users/djacques/miniconda3/envs/kedro_iris_ml_ops/lib/python3.8/site-packages/kedro/framework/context/context.py:59: DeprecationWarning: Accessing io via the context will be deprecated in Kedro 0.18.0.
  warn(


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [2]:
# Run the training pipeline
session.run(pipeline_name="train")

2021-08-08 14:59:27,505 - root - INFO - ** Kedro project kedro_iris_ml_ops
2021-08-08 14:59:27,556 - kedro.io.data_catalog - INFO - Loading data from `example_iris_data` (CSVDataSet)...
2021-08-08 14:59:27,576 - kedro.io.data_catalog - INFO - Loading data from `params:example_test_data_ratio` (MemoryDataSet)...
2021-08-08 14:59:27,578 - kedro.pipeline.node - INFO - Running node: split: split_data([example_iris_data,params:example_test_data_ratio]) -> [example_test_x,example_test_y,example_train_x,example_train_y]
2021-08-08 14:59:27,621 - kedro.io.data_catalog - INFO - Saving data to `example_train_x` (MemoryDataSet)...
2021-08-08 14:59:27,623 - kedro.io.data_catalog - INFO - Saving data to `example_train_y` (MemoryDataSet)...
2021-08-08 14:59:27,626 - kedro.io.data_catalog - INFO - Saving data to `example_test_x` (MemoryDataSet)...
2021-08-08 14:59:27,630 - kedro.io.data_catalog - INFO - Saving data to `example_test_y` (MemoryDataSet)...
2021-08-08 14:59:27,636 - kedro.runner.sequenti

{}

In [7]:
import pandas as pd
from kedro.framework.session import KedroSession

# Create a pandas dataframe with new observations to predict
predict_input = pd.DataFrame({
    "sepal_length": [5.1, 5.0, 4.5, 4.8, 6.2, 1],
    "sepal_width" : [3.2, 3.3, 3.5, 3.7, 3.2, 1],
    "petal_length": [1.3, 1.3, 1.4, 1.4, 5.2, 1],
    "petal_width": [0.2, 0.2, 0.2, 0.2, 2.0, 1]
})

predict_input

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.2,1.3,0.2
1,5.0,3.3,1.3,0.2
2,4.5,3.5,1.4,0.2
3,4.8,3.7,1.4,0.2
4,6.2,3.2,5.2,2.0
5,1.0,1.0,1.0,1.0


In [11]:
# Close an existing session
session.close()

# Execute our prediction pipeline on new data:
with KedroSession.create(package_name="kedro_iris_ml_ops", project_path="../", extra_params={"prediction_input": predict_input}) as session:
    
    session.run(pipeline_name="predict")

2021-08-08 15:05:04,079 - kedro.framework.session.store - INFO - `save()` not implemented for `BaseSessionStore`. Skipping the step.
2021-08-08 15:05:04,206 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-08-08 15:05:04,257 - root - INFO - ** Kedro project kedro_iris_ml_ops
2021-08-08 15:05:04,275 - kedro.io.data_catalog - INFO - Loading data from `params:prediction_input` (MemoryDataSet)...
2021-08-08 15:05:04,277 - kedro.pipeline.node - INFO - Running node: predict_input_validation([params:prediction_input]) -> [predict_df]
2021-08-08 15:05:04,282 - kedro.io.data_catalog - INFO - Saving data to `predict_df` (MemoryDataSet)...
2021-08-08 15:05:04,284 - kedro.runner.sequential_runner - INFO - Completed 1 out of 2 tasks
2021-08-08 15:05:04,288 - kedro.io.data_catalog - INFO - Loading data from `example_model` (PickleDataSet)...
2021-08-08 15:05:04,293 - kedro.io.data_catalog - INFO - Loading data from `predict_df` (Mem

In [12]:
# Extract predictions from our catalog
session.load_context().catalog.load("predictions")

2021-08-08 15:06:17,244 - kedro.io.data_catalog - INFO - Loading data from `predictions` (PickleDataSet)...


array([0, 0, 0, 0, 2, 2])